In [1]:
import bs4 as BeautifulSoup
import urllib.request
import pandas as pd

main_url = 'https://www.basketball-reference.com/players/'
letters = 'abcdefghijklmnopqrstuvwxyz'
letters_list = list(letters)

In [4]:
test = urllib.request.urlopen('https://www.basketball-reference.com/players/a/')